In [1]:
from AppModules.Functions.Basic_Functions import *
from AppModules.Data_Processing.Data_Functions import *
from AppModules.File_Handling.VP_Handling import *
from AppModules.File_Handling.CSV_Handling import *

import AppModules.Data_Processing.Poly_Viewer as Poly
import AppModules.Data_Processing.Smooth_Viewer as Smooth
import AppModules.Data_Processing.Baseline as Baseline
import AppModules.Data_Processing.Plotting_Processed_Data as Processed

import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter as savgol
import pandas as pd

import gc

# Constants
HOME_FOLDER = os.getcwd()

VP_FOLDER  = 'VP Data'
CSV_FOLDER = 'VP Data (CSV)'


TUNNEL_COLOR = 'blue'
DISS_COLOR, FREQ_COLOR = 'k', 'r'
COLORS = (DISS_COLOR, FREQ_COLOR)


WOO, HAT = '\U0001F389', '\U0001F973'

#### Finding and Grouping ".vpdata" Files -- Necessary

In [2]:
DATA_FOLDER = GetFolder() 
print(f'Chosen Data Folder: {DATA_FOLDER}')
os.chdir(DATA_FOLDER)

CreateFolders(VP_FOLDER, CSV_FOLDER)
Group_VP(VP_FOLDER)

print(f'VP Files found [{len(os.listdir(VP_FOLDER))}]')

os.chdir(HOME_FOLDER)
timestamp()

Chosen Data Folder: C:/Users/Work/Downloads/240117 Binod Data/Binod Data (1)
Unsorted VP Files Found: Attempting to Move [9] Files


100%|██████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 1955.49it/s]

VP Files found [9]

Executed: 2024/01/20 03:46:42 PM


#### Choosing a ".vpdata" file to extract data into one/two CSV file(s) for further processing

In [5]:
FILE_PATH = GetFile(WindowName='Select .vpdata File')
DATA_FOLDER = os.path.dirname(os.path.dirname(FILE_PATH))
os.chdir(DATA_FOLDER)

FILE = os.path.basename(FILE_PATH)
print(f'Chosen VP File: {FILE}')
if Check_CSV(CSV_FOLDER, FILE): 
    print('CSV File(s) Already Created --> !! CSV File not Created')
    
else: 
    Make_CSV(f'{VP_FOLDER}{os.path.sep}{FILE}', CSV_FOLDER)
    print(f'CSV File(s) Created --> Move on to Data Processing')
    
os.chdir(HOME_FOLDER)
timestamp()

Chosen VP File: Pt dot probe000-VP027-VP 1.vpdata
CSV File(s) Created --> Move on to Data Processing

Executed: 2024/01/20 03:49:00 PM


# █████  DATA PROCESSING: START HERE  █████

## ►► Choosing a ".csv" file to extract data and process

In [6]:
FILE_PATH = GetFile()
DATA_FOLDER = os.path.dirname(os.path.dirname(FILE_PATH))
os.chdir(DATA_FOLDER)

FILE = os.path.basename(FILE_PATH)
print(f'Chosen CSV File: {FILE} \n')
print(f'--> {FILE_PATH}')

GetHeaders_CSV(FILE_PATH, LIST=True)

##################################

%matplotlib qt

plt.close('all')
gc.collect()

timestamp()

Chosen CSV File: Pt dot probe000-VP027-VP 1_Trace.csv 

--> C:/Users/Work/Downloads/240117 Binod Data/Binod Data (1)/VP Data (CSV)/Pt dot probe000-VP027-VP 1_Trace.csv

Headers Found in File:

	- Bias (V)
	- ADC1-SP (V)
	- ADC2-Mx2 (V)
	- ADC4 (V)

Executed: 2024/01/20 03:49:14 PM


## ► Removing polynomial background from data (and saving in the ".csv" file)

In [7]:
# Create plots for visualizing and choosing degrees
Poly.PolyFit(FILE_PATH, COLORS)

Polynomial Degrees Used: {'diss': 4, 'freq': 4}

Executed: 2024/01/20 03:49:36 PM


In [8]:
# Removing Background 
plt.close('all')
gc.collect()

Poly.RemoveBackground(FILE_PATH)
print('Polynomial Background Removed - Degrees Used:', Poly.DEG)

timestamp()

Polynomial Background Removed - Degrees Used: {'diss': 4, 'freq': 4}

Executed: 2024/01/20 03:49:38 PM


## ► Smoothing data and processing data (and saving in the ".csv" file)

In [9]:
# Create plots for visualizing and choosing smoothing parameters
Smooth.SmoothFit(FILE_PATH, COLORS)

Smoothing Windows Used: {'diss': 35, 'freq': 48}

Executed: 2024/01/20 03:50:06 PM


In [10]:
# Smoothing Data 
plt.close('all')
gc.collect()

Smooth.SmoothData(FILE_PATH)
print('Data Smoothed - Parameters Used:', Smooth.W)

timestamp()

Data Smoothed - Parameters Used: {'diss': 35, 'freq': 48}

Executed: 2024/01/20 03:50:08 PM


In [12]:
# Create plots for visualizing and removing baseline
dissL, dissP, dissNiter = 20000000, 0.000005, 100  # Follow shape
freqL, freqP, freqNiter = 30000000, 0.00005, 100
DISS_PARAMS, FREQ_PARAMS  = (dissL, dissP, dissNiter), (freqL, freqP, freqNiter)

CORRECT_BASELINE = False

baseDISS, baseFREQ = Baseline.BaselineFit(DISS_PARAMS, FREQ_PARAMS, FILE_PATH, COLORS, CORRECT_BASELINE)

print('Baseline Plotted')
timestamp()

Baseline Plotted

Executed: 2024/01/20 03:50:20 PM


In [13]:
# Removing Baseline 
plt.close('all')
gc.collect()

if(CORRECT_BASELINE): 
    Baseline.RemoveBaseline(DISS_PARAMS, FREQ_PARAMS, FILE_PATH)
    print('Data Baseline Removed | Parameters Used:\n\n\tdissL, dissP, dissNiter =', DISS_PARAMS, 
                                                    '\n\tfreqL, freqP, freqNiter =', FREQ_PARAMS )

else: print('No Baseline Removed')
    
timestamp()

No Baseline Removed

Executed: 2024/01/20 03:50:29 PM


## ► Plotting processed data

In [14]:
# Plotting data after processing, mainly for visualizing and publishing
df = pd.read_csv(FILE_PATH)
BIAS, DISS, FREQ = Processed.GetData(df)

# Unit Conversion of Data
freqConvert = 100                # [Hz/V]
FREQ        = -freqConvert*FREQ  # [Hz/V]*[V] = [Hz]

f0    = 160000       # [Hz]
Q     = 100000       # []
omega = (2*np.pi*f0) # [Hz]/[] = [Hz]

DEG      = 4  # Fitting original data, creating polynomial, evaluating at BIAS, and finding minimum
diss_min = min( np.poly1d( np.polyfit(df['Bias (V)'], df['ADC4 (V)'], deg=DEG ) )(BIAS) ) 
DISS     = omega/Q*(DISS/diss_min) # [Hz]*([]*[V]/[V]) = [Hz]


# Final Processing
a, b = 71, 2 
DISS, FREQ = savgol(DISS, a, b), savgol(FREQ, a, b)
Processed.NewRange(BIAS, DISS, FREQ, left=-9, right=10)

    
# Plots  
print(f'Plotting processed data for: {FILE}')
fig, (axDISS, axFREQ) = plt.subplots(2, figsize= (2, 1), sharex=True)
Processed.PlotSetup( (fig, axDISS, axFREQ, FILE, FREQ_COLOR) )

lw = 2
axDISS.plot(BIAS, DISS, label='Diss.', color= DISS_COLOR, linewidth=lw )
axFREQ.plot(BIAS, FREQ, label='Freq.', color= FREQ_COLOR, linewidth=lw )



#axDISS.legend(loc='upper left')
#axFREQ.legend(loc='lower left')

plt.show()

print( f'Data Plotted! {WOO}{HAT}{WOO}')
timestamp()

Plotting processed data for: Pt dot probe000-VP027-VP 1_Trace.csv
Data Plotted! 🎉🥳🎉

Executed: 2024/01/20 03:50:31 PM
